<a href="https://colab.research.google.com/github/ikanx101/G-Colab/blob/main/Kelas_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [313]:
library(dplyr)
library(ompr)
library(ompr.roi)
library(ROI.plugin.glpk)

In [314]:
rm(list=ls())
n = 55
m = 4 # ada 4 kelas
pilih = 3 # pilih 3 batch
capacity = rep.int(round(n/m,0)+1,m)

set.seed(101)
pilihan = list()
for(i in 1:n){
  pilihan[i] = list(sample(m,pilih))
}

pilihan

[[1]]
[1] 1 4 2

[[2]]
[1] 3 4 1

[[3]]
[1] 2 3 1

[[4]]
[1] 4 3 2

[[5]]
[1] 3 4 2

[[6]]
[1] 1 3 4

[[7]]
[1] 2 3 4

[[8]]
[1] 3 1 4

[[9]]
[1] 1 2 3

[[10]]
[1] 1 4 3

[[11]]
[1] 2 4 3

[[12]]
[1] 1 2 3

[[13]]
[1] 2 3 1

[[14]]
[1] 1 4 2

[[15]]
[1] 1 2 3

[[16]]
[1] 4 2 1

[[17]]
[1] 4 3 2

[[18]]
[1] 2 1 4

[[19]]
[1] 1 4 3

[[20]]
[1] 3 4 2

[[21]]
[1] 1 4 3

[[22]]
[1] 2 4 1

[[23]]
[1] 1 2 3

[[24]]
[1] 1 2 3

[[25]]
[1] 2 1 4

[[26]]
[1] 1 2 4

[[27]]
[1] 2 1 4

[[28]]
[1] 2 4 3

[[29]]
[1] 2 1 4

[[30]]
[1] 1 2 3

[[31]]
[1] 1 2 4

[[32]]
[1] 1 3 4

[[33]]
[1] 1 2 3

[[34]]
[1] 3 1 4

[[35]]
[1] 4 2 3

[[36]]
[1] 4 3 2

[[37]]
[1] 4 2 3

[[38]]
[1] 3 1 4

[[39]]
[1] 1 4 3

[[40]]
[1] 2 3 1

[[41]]
[1] 2 4 1

[[42]]
[1] 2 1 4

[[43]]
[1] 1 2 4

[[44]]
[1] 2 1 3

[[45]]
[1] 2 3 1

[[46]]
[1] 2 4 1

[[47]]
[1] 4 3 1

[[48]]
[1] 3 4 1

[[49]]
[1] 2 3 1

[[50]]
[1] 4 1 3

[[51]]
[1] 3 4 1

[[52]]
[1] 4 3 1

[[53]]
[1] 4 2 3

[[54]]
[1] 1 2 4

[[55]]
[1] 4 3 1

In [315]:
bobot = function(pilihan,course){
  pilihan = unlist(pilihan)
  ikang = which(pilihan == course)
  ikang = ifelse(length(ikang) == 0, -1000, 4-ikang)
  return(ikang)
}
pilihan[1]
bobot(pilihan[1],1)

[[1]]
[1] 1 4 2

[1] 3

In [316]:
model <- MIPModel() %>%
  
  # 1 iff student i is assigned to course m
  add_variable(x[i, j], i = 1:n, j = 1:m, type = "binary") %>%
  
  # maximize the preferences
  set_objective(sum_expr(bobot(pilihan[i], j) * x[i, j], i = 1:n, j = 1:m),
                "max") %>%
  
  # we cannot exceed the capacity of a course
  add_constraint(sum_expr(x[i, j], i = 1:n) <= capacity[j], j = 1:m) %>% 
  
  # each student needs to be assigned to one course
  add_constraint(sum_expr(x[i, j], j = 1:m) == 1, i = 1:n)
model

Mixed integer linear optimization problem
Variables:
  Continuous: 0 
  Integer: 0 
  Binary: 220 
Model sense: maximize 
Constraints: 59 

In [317]:
result <- solve_model(model, with_ROI(solver = "glpk", verbose = TRUE))
result

<SOLVER MSG>  ----
GLPK Simplex Optimizer, v4.65
59 rows, 220 columns, 440 non-zeros
      0: obj =  -0.000000000e+00 inf =   5.500e+01 (55)
     58: obj =  -1.090300000e+04 inf =   0.000e+00 (0)
*   180: obj =   1.590000000e+02 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Integer Optimizer, v4.65
59 rows, 220 columns, 440 non-zeros
220 integer variables, all of which are binary
Integer optimization begins...
Long-step dual simplex will be used
+   180: mip =     not found yet <=              +inf        (1; 0)
+   180: >>>>>   1.590000000e+02 <=   1.590000000e+02   0.0% (1; 0)
+   180: mip =   1.590000000e+02 <=     tree is empty   0.0% (0; 1)
INTEGER OPTIMAL SOLUTION FOUND
<!SOLVER MSG> ----


Status: optimal
Objective value: 159

In [319]:
final = 
  result %>% 
  get_solution(x[i,j]) %>%
  filter(value > 0) %>%
  rename(id_peserta = i,
         batch_ke = j) %>%
  select(-variable,-value)

final
table(final$batch_ke)

id_peserta,batch_ke
<int>,<int>
9,1
10,1
12,1
14,1
15,1
19,1
23,1
24,1
26,1



 1  2  3  4 
15 15 10 15 

In [ ]:
#system("sudo apt-get install libglpk-dev")
#install.packages("ROI.plugin.glpk")